In [44]:
import random
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import tz

In [69]:
def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

# fume_query(target="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", server="biotech_main", start="2021-12-25T00:00:00.000", end="2021-12-25T00:00:00.000")
# fume_query(target="#biotech_weather/temperature_tn", server="biotech_main", start = str(datetime(2021, 11, 17, 1)), end = str(datetime(2021, 11, 17, 2)))

def query_to_list(point, server, start, end):
    master = fume_query(point, server, start, end)

    list = pd.Series(data=[i[0] for i in master], index=[i[1] for i in master])
    # print("\n", point, "\n", list)

    list = list[~list.index.duplicated()]
    # print("\n", point, " new\n", list)

    return list

In [81]:
def total_time_sash_open(sash_point, occ_point, server, start, end, is_occupied):
  sash_list = query_to_list(sash_point, server, start, end)
  occ_list = query_to_list(occ_point, server, start, end)

  df = pd.concat([sash_list, occ_list], axis=1)
  df.columns = ["sash", "occ"]
  # display(df)

  time_interval = df.index[1].minute - df.index[0].minute
  # print("Time interval", time_interval)

  # Figure out closed sash position
  # display(df["sash"].value_counts())

  # from running the above on a large time difference, 1.2 inches is the most common smallest value
  df["time_open_mins"] = np.where((df["sash"] > 1.2), time_interval, 0)

  df = df.dropna()
  # df.index = df.index.map(lambda x: x.to_pydatetime().replace(tzinfo=tz.tzutc()).astimezone(tz.tzlocal()))
  # display(df)

  df = df[df['occ']==int(is_occupied)]

  # display(df)

  df = df.groupby(pd.Grouper(freq='60Min', label='right')).sum()

  df.index = df.index.strftime('%s').astype("int")


  return df["time_open_mins"]

In [90]:
def compute_value(_hood, timestamp):
    """This would be replaced by a real algorithm that computed values 
    based on which hood, the environment, etc..."""
    return [random.random(), timestamp]

sash_data_occ = total_time_sash_open(sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
                                occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
                                server="biotech_main",
                                start = str(datetime(2021, 11, 17)),
                                end = str(datetime(2021, 11, 18)),
                                is_occupied=True)

sash_data_unocc = total_time_sash_open(sash_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash",
                                occ_point="#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
                                server="biotech_main",
                                start = str(datetime(2021, 11, 17)),
                                end = str(datetime(2021, 11, 18)),
                                is_occupied=False)

# print(sash_data_occ)
# print(sash_data_unocc)

# final_df = pd.DataFrame(data={"occ": sash_data_occ, "unocc" : sash_data_unocc})
# final_df = final_df.fillna(0)
# final_df

# the x-api-key is the secret authentication to allow posting updates
headers = {"x-api-key": "LfSpEZ3woOuljzb2EAg0"}
# the endpoint URL for trend updates
TREND_API_URL = "https://7zhs2wplt9.execute-api.us-east-1.amazonaws.com/dev/trend"
# time in seconds that the request will timeout if no response is received.
# the API will only run for 6 seconds, so if it can't process
# the data posted in that amount of time it will fail.
API_TIMEOUT = 6

# timestamps
now = int(time.time())
last_hour = now - now % 3600
start_time = last_hour - (3600 * 6)

# a list of points
points = [
    # "Biotech.Floor_4.Lab_433.Hood_1.energy.occ",
    # "Biotech.Floor_4.Lab_433.Hood_1.energy.unocc",
    "Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.occ",
    "Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.unocc",
    # "Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc",
    # "Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTime.occ",
]

# insert some data and log the results
for point in points:
    # data = [compute_value(point, ts) for ts in range(start_time, last_hour, 3600)]

    data = np.fliplr(sash_data_occ.reset_index().to_numpy()).tolist()
    print(data)

    json_obj = {"object_name": point, "data": data }
    # e.g.
    # { "object_name": "Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ",
    # "data": [[0, 1681462800], [0, 1681466400], [0, 1681470000], [4.430475, 1681473600]],
    # }
    req_start = time.time()
    response = requests.post(TREND_API_URL, json=json_obj, headers=headers, timeout=API_TIMEOUT)
    req_end = time.time()
    r = response.json()
    print(f"""received status_code:{response.status_code} msg:{r.get('message', None)}
    rows:{r.get('affectedRows', None)} duplicates:{r.get('duplicates', None)} 
    for point:{point} with id:{r.get('obj_id', None)} in {req_end-req_start:.3f} seconds.""")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
[[60, 1637146800], [50, 1637150400], [10, 1637154000], [60, 1637157600], [30, 1637161200], [20, 1637164800], [45, 1637168400], [20, 1637172000], [50, 1637175600], [60, 1637179200], [60, 1637182800], [60, 1637186400], [60, 1637190000], [60, 1637193600], [60, 1637197200], [60, 1637200800], [60, 1637204400], [60, 1637208000], [60, 1637211600], [60, 1637215200], [60, 1637218800], [5, 1637222400], [35, 1637226000], [30, 1637229600], [0, 1637233200], [0, 1637236800], [0, 1637240400], [0, 1637244000], [0, 1637247600], [0, 1637251200], [30, 1637254800], [40, 1637258400], [60, 1637262000], [60, 1637265600], [60, 1637269200], [60, 1637272800], [60, 1637276400], [60, 1637280000], [60, 1637283600], [60, 1637287200], [60, 1637290800], [60, 1637294400], [60, 1637298000], [5, 1637301600]]
received status_code:200 msg:Insert
    rows:44 duplicates:44 
    for point:Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.occ with id:11233556 in 0.

In [89]:
def synthetic_query(target, start, end):
    url = "https://portal.emcs.cornell.edu/api/datasources/proxy/5/query"
    data = {
        "range": {
          "from": start,
          "to": end,
        },
        "targets": [
          {
            "target": target
          }
        ],

      }
    request = requests.post(url, json=data)
    print(request)
    # print(request.json())
    return create_tuple(request)

display(synthetic_query(target="Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.occ", start=str(datetime(2021, 11, 16)), end=str(datetime(2021, 11, 20))))
display(synthetic_query(target="Biotech.Floor_4.Lab_433.Hood_1.sashOpenTime.unocc", start=str(datetime(2021, 11, 16)), end=str(datetime(2021, 11, 20))))

<Response [200]>


array([(60., '2021-11-17T06:00:00.000'), (50., '2021-11-17T07:00:00.000'),
       (10., '2021-11-17T08:00:00.000'), (60., '2021-11-17T09:00:00.000'),
       (30., '2021-11-17T10:00:00.000'), (60., '2021-11-17T11:00:00.000'),
       (50., '2021-11-17T12:00:00.000'), (10., '2021-11-17T13:00:00.000'),
       (60., '2021-11-17T14:00:00.000'), (30., '2021-11-17T15:00:00.000'),
       (20., '2021-11-17T16:00:00.000'), (45., '2021-11-17T17:00:00.000'),
       (20., '2021-11-17T18:00:00.000'), (50., '2021-11-17T19:00:00.000'),
       (60., '2021-11-17T20:00:00.000'), (60., '2021-11-17T21:00:00.000'),
       (60., '2021-11-17T22:00:00.000'), (60., '2021-11-17T23:00:00.000'),
       (60., '2021-11-18T00:00:00.000'), (60., '2021-11-18T01:00:00.000'),
       (60., '2021-11-18T02:00:00.000'), (60., '2021-11-18T03:00:00.000'),
       (60., '2021-11-18T04:00:00.000'), (60., '2021-11-18T05:00:00.000'),
       (60., '2021-11-18T06:00:00.000'), (60., '2021-11-18T07:00:00.000'),
       ( 5., '2021-11-18T

<Response [200]>


array([(60., '2021-11-17T06:00:00.000'), (50., '2021-11-17T07:00:00.000'),
       (10., '2021-11-17T08:00:00.000'), (60., '2021-11-17T09:00:00.000'),
       (30., '2021-11-17T10:00:00.000'), (60., '2021-11-17T11:00:00.000'),
       (50., '2021-11-17T12:00:00.000'), (10., '2021-11-17T13:00:00.000'),
       (60., '2021-11-17T14:00:00.000'), (30., '2021-11-17T15:00:00.000'),
       (20., '2021-11-17T16:00:00.000'), (45., '2021-11-17T17:00:00.000'),
       (20., '2021-11-17T18:00:00.000'), (50., '2021-11-17T19:00:00.000'),
       (60., '2021-11-17T20:00:00.000'), (60., '2021-11-17T21:00:00.000'),
       (60., '2021-11-17T22:00:00.000'), (60., '2021-11-17T23:00:00.000'),
       (60., '2021-11-18T00:00:00.000'), (60., '2021-11-18T01:00:00.000'),
       (60., '2021-11-18T02:00:00.000'), (60., '2021-11-18T03:00:00.000'),
       (60., '2021-11-18T04:00:00.000'), (60., '2021-11-18T05:00:00.000'),
       (60., '2021-11-18T06:00:00.000'), (60., '2021-11-18T07:00:00.000'),
       ( 5., '2021-11-18T